<a href="https://colab.research.google.com/github/shinhs0920/19-lab/blob/master/GAN_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 생성적 적대 신경망(**GAN**)

### 개요

GAN은 **생성기**와 **판별기**라는 두 네트워크로 구성된다. 

**생성기**의 역할 : 판별기를 속일 수 있는 위조 데이터(가짜 데이터)를 계속 생성한다.
<BR>**판별기**의 역할 : 생성기로부터 받은 **위조 데이터**와 실제 샘플 데이터에서 받은 **실제 데이터**를 구분한다.

생성기의 입력은 노이즈이며, 출력은 합성된 가짜 신호이다.
<BR>판별기의 입력은 진짜 신호 혹은 생성기의 출력인 가짜 신호(합성 신호)이다.

유효 신호의 레이블은 1.0(즉, 진짜 데이터)이며, 합성된 신호의 레이블은 모두 0.0이다.

### GAN 모델 데이터 처리 과정

자기가 만든 출력이 진짜 데이터인 척하며 GAN에게 자기의 출력에 1.0으로 레이블을 붙일 것을 요청한다. 그리고 1.0으로 레이블 붙여진 가짜 데이터를 판별기에게 보이면 판별기에 의해 그 데이터는 레이블이 0.0에 가까워지며 가짜 신호로 분류된다.

또한 판별기는 새로운 데이터 훈련 시, 자신이 예측한 값을 포함하기에 GAN은 오차값을 염두해 둔다.이 경사를 판별기의 마지막 층에서 생성기의 첫 번째 층으로 역전파시킨다.

## 심층 합성곱 GAN(DCGAN)

-> 초기에 심층 CNN을 사용해 GAN을 구현한 모델 (가짜 이미지를 생성하기 위해 사용된다.)

### 모델 설계 시 차이점

- MaxPooling2D 대신 strides>1 합성곱을 사용한다. CNN이 특징 맵 크기를 조절하는 법을 학습한다.

- Dense 계층은 생성기의 첫번째 계층에서 z-vector를 받기 위해 사용된다. 

- 각 계층의 입력이 평균 0의 분산을 가지기 위해 배치 정규화(BN)을 사용한다.

- 판별기 전 계층에 Leaky ReLU를 사용한다. 이는 ReLU와 달리 $alpha\times input $처럼 작은 경사 값을 생성하는 함수이다. 

### 모델 구성

생성기는 -1.0에서 1.0 사이에서 생성된 100차원 z-벡터를 받는다. 첫번째 계층은 $7\times 7\times 128=6,272$개의 유닛을 가진 Dense 계층이다. 유닛 개수는 최종 출력 이미지의 차원 ($28\times 28\times 1$, 28은 7의 배수)과 첫 번째 Conv2DTranspose의 필터 개수 128을 기준으로 한다.

- Conv2DTranspose : Conv2D의 전치이며, 특징 맵에서 이미지를 생산한다.

두 번의 Conv2DTranspose 계층을 거치면 특징 맵 크기는 $28\times 28\times$ '필터 갯수'이다. 마지막 계층에는 가짜 이미지를 생성하는 시그모이드 함수가 있다. <br>각 픽셀은 [0,255]의 회색조 레이블에 대응하는 [0.0, 1.0]으로 정규화 된다.

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.keras.layers import Activation, Dense, Input
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.layers import Reshape, Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import load_model

import numpy as np
import math
import matplotlib.pyplot as plt
import os
import argparse

# 1. 생성기 모델 구성
def build_generator(inputs, image_size):
 # 가짜 이미지를 만들기 위해 BN-ReLU-Conv2DTranpose 으로 구성. 출력층 활성화 함수는 tanh 대신 Sigmoid 사용 - Sigmoid가 더 수렴이 쉬움
 # 두개의 층을 각각 지나면서 크기가 절반씩 줄어드므로 총 크기는 1/4 이 된다   
    image_resize = image_size // 4

   
# 네트워크 매개변수
    kernel_size = 5
    layer_filters = [128, 64, 32, 1]

    x = Dense(image_resize * image_resize * layer_filters[0])(inputs)
    x = Reshape((image_resize, image_resize, layer_filters[0]))(x)

    for filters in layer_filters:
         # 첫 두 합성곱 계층이 strides = 2 사용
        # 마지막 두 계층은 strides = 1 사용
        if filters > layer_filters[-2]:
            strides = 2
        else:
            strides = 1
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Conv2DTranspose(filters=filters,
                            kernel_size=kernel_size,
                            strides=strides,
                            padding='same')(x)

    x = Activation('sigmoid')(x)
    generator = Model(inputs, x, name='generator')
    return generator

# 2. 판별기 모델 구성
def build_discriminator(inputs):
     # 진짜와 가짜를 구분하기 위해 LeakyReLU-Conv2D 구조 사용. 배치 정규화로는 수렴하지 않으므로 사용하지 않음.

    kernel_size = 5
    layer_filters = [32, 64, 128, 256]

    x = inputs
    for filters in layer_filters:
          # 첫 3개의 합성곱 계층은 strides = 2 사용.
        # 마지막 계층은 strides = 1 사용.
        if filters == layer_filters[-1]:
            strides = 1
        else:
            strides = 2
        x = LeakyReLU(alpha=0.2)(x)
        x = Conv2D(filters=filters,
                   kernel_size=kernel_size,
                   strides=strides,
                   padding='same')(x)

    x = Flatten()(x)
    x = Dense(1)(x)
    x = Activation('sigmoid')(x)
    discriminator = Model(inputs, x, name='discriminator')
    return discriminator

# 5. 판별기와 적대적 네트워크 훈련

def train(models, x_train, params):
 # 판별기는 진짜와 가짜 이미지를 가지고 훈련되며, 그 후 적대적 네트워크는 진짜인 척 하는 가짜 이미지로 훈련된다
    # GAN 모델  
    generator, discriminator, adversarial = models

   # 네트워크 파라미터
    batch_size, latent_size, train_steps, model_name = params
    
   # 500단계마다 생성기 이미지가 저장된다.
    save_interval = 500
   # 노이즈 벡터 - 훈련 기간 동안 생성기 출력 이미지 변화 관찰 위한 벡터
    noise_input = np.random.uniform(-1.0, 1.0, size=[16, latent_size])
   
    train_size = x_train.shape[0]
    for i in range(train_steps):
        # 1배치에 대한 판별기 훈련
        # 1배치 = 진짜(label=1.0)와 가짜 이미지(label=0.0)로 구성된 배치
        # 데이터 세트에서 임의로 진짜 이미지 선택
        rand_indexes = np.random.randint(0, train_size, size=batch_size)
        real_images = x_train[rand_indexes]
       
        noise = np.random.uniform(-1.0,
                                  1.0,
                                  size=[batch_size, latent_size])
        # 가짜 이미지 생성
        fake_images = generator.predict(noise)

        # 진짜 + 가짜 이미지 = 훈련 데이터의 1 배치
        x = np.concatenate((real_images, fake_images))

        # 진짜와 가짜 이미지에 레이블을 붙인다. 진짜는 1.0
        y = np.ones([2 * batch_size, 1])

        # 가짜는 0.0
        y[batch_size:, :] = 0.0

        # 판별기 네트워크 훈련, 손실과 정확도 기록
        loss, acc = discriminator.train_on_batch(x, y)
        log = "%d: [discriminator loss: %f, acc: %f]" % (i, loss, acc)

 # 6. 1 배치에 대한 적대적 네트워크 훈련
        # label=1.0인 가짜 이미지로 구성된 1배치
        # 적대적 네트워크에서 판별기 가중치가 고정되므로 생성기만 훈련된다.
    
        noise = np.random.uniform(-1.0,
                                  1.0, 
                                  size=[batch_size, latent_size])
        y = np.ones([batch_size, 1])
        # 적대적 네트워크 훈련 
        # 가짜 이미지는 분류를 위해 적대적 네트워크의 판별기 입력으로 전달된다.
        # 손실, 정확도 기록
        loss, acc = adversarial.train_on_batch(noise, y)
        log = "%s [adversarial loss: %f, acc: %f]" % (log, loss, acc)
        print(log)
        if (i + 1) % save_interval == 0:
            plot_images(generator,
                        noise_input=noise_input,
                        show=False,
                        step=(i + 1),
                        model_name=model_name)
   
    generator.save(model_name + ".h5")
    
# 3. 훈련
def build_and_train_models():
    (x_train, _), (_, _) = mnist.load_data()

    # CNN을 위해 (28, 28, 1) 로 데이터 형상 구현 및 정규화
image_size = x_train.shape[1]
x_train = np.reshape(x_train, [-1, image_size, image_size, 1])
x_train = x_train.astype('float32') / 255

model_name = "dcgan_mnist"
    
latent_size = 100 # z-vector는 100차원 벡터
batch_size = 64
train_steps = 40000
lr = 2e-4
decay = 6e-8
input_shape = (image_size, image_size, 1)

    # 판별기 모델 구성
inputs = Input(shape=input_shape, name='discriminator_input')
discriminator = build_discriminator(inputs)
  
    # RMSprop로 최적화 - 쉽게 수렴한다.
optimizer = RMSprop(lr=lr, decay=decay)
discriminator.compile(loss='binary_crossentropy',
                        optimizer=optimizer,
                          metrics=['accuracy'])
discriminator.summary()

    # 생성기 모델 구성
input_shape = (latent_size, )
inputs = Input(shape=input_shape, name='z_input')
generator = build_generator(inputs, image_size)
generator.summary()

    # 적대적 모델 구성
optimizer = RMSprop(lr=lr * 0.5, decay=decay * 0.5)
  
    # 적대적 모델 훈련 과정 동안 판별기 가중치 고정
discriminator.trainable = False

    # adversarial = generator + discriminator (생성기 판별기 결합)
adversarial = Model(inputs, 
                        discriminator(generator(inputs)),
                        name=model_name)
adversarial.compile(loss='binary_crossentropy',
                        optimizer=optimizer,
                        metrics=['accuracy'])
adversarial.summary()

    # 생성기와 판별기 네트워크 훈련
models = (generator, discriminator, adversarial)
params = (batch_size, latent_size, train_steps, model_name)
train(models, x_train, params)

NameError: ignored

판별기의 입력은 $28\times 28\times 1$ MNIST 이미지이며, 진짜(1.0) 혹은 가짜(0.0)으로 분류된다. 또한 판별기는 4개의 CNN 계층을 갖는다. 또한 최종 출력 계층은 일차원으로 변환되어 시그모이드함수를 거쳐 0.0~1.0 사이 값으로 예측을 생성한다.

이렇게 생성기와 판별기 계층을 구성한 뒤 둘을 결합해 적대적 모델을 구성한다.
적대적 모델의 최적화는 RMSprop 최적화를 사용한다. 적대적 신경망의 학습 속도를 판별기 학습 속도의 반으로 설정하면 더 안정적으로 훈련할 수 있다.

생성기는 적대적 네트워크를 통해 훈련된다. 먼저 임의로 데이터 세트에서 실제 이미지로 구성된 배치를 선택하여 1.0 레이블을 붙이며, 생성기에서 가짜 이미지로 구성된 배치가 생성된다. 이는 0.0 레이블이 붙으며 이 두 배치를 연결하여 판별기 훈련에 사용한다.

위 과정이 완료되면 생성기가 새로운 가짜 이미지 배치를 생성하고 거기에 진짜 레이블인 1.0을 붙여 적대적 네트워크를 훈련시키기 위해 사용한다.

## 조건부 GAN (CGAN)

DCGAN에서 가짜 이미지는 임의로 생성된다. 생성기에서 어떤 숫자를 생성할지 우리가 제어할 수 없다. 이 문제를 보완한 것이 조건부 GAN이다.

동일한 GAN을 사용하지만 생성기와 판별기 입력 모두에 조건을 부여한다. 그 조건은 입력을 원-핫 벡터 형태로 변환하는 것이다. DCGAN과의 차이점은 입력으로 원-핫 벡터를 추가로 받는 것이다.

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.keras.layers import Activation, Dense, Input
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.layers import Reshape, Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

import numpy as np
import math
import matplotlib.pyplot as plt
import os
import argparse

def build_generator(inputs, labels, image_size):
    
    image_resize = image_size // 4
   
    kernel_size = 5
    layer_filters = [128, 64, 32, 1]

    x = concatenate([inputs, labels], axis=1) # DCGAN과 달리 concatenate가 필요하다.
    x = Dense(image_resize * image_resize * layer_filters[0])(x)
    x = Reshape((image_resize, image_resize, layer_filters[0]))(x)

    for filters in layer_filters:
        
        if filters > layer_filters[-2]:
            strides = 2
        else:
            strides = 1
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Conv2DTranspose(filters=filters,
                            kernel_size=kernel_size,
                            strides=strides,
                            padding='same')(x)

    x = Activation('sigmoid')(x)
    generator = Model([inputs, labels], x, name='generator') # 입력은 y_labes에 의해 조건이 부여된다.
    return generator

def build_discriminator(inputs, labels, image_size):
  
    kernel_size = 5
    layer_filters = [32, 64, 128, 256]

    x = inputs

    y = Dense(image_size * image_size)(labels)
    y = Reshape((image_size, image_size, 1))(y)
    x = concatenate([x, y])

    for filters in layer_filters:
        # first 3 convolution layers use strides = 2
        # last one uses strides = 1
        if filters == layer_filters[-1]:
            strides = 1
        else:
            strides = 2
        x = LeakyReLU(alpha=0.2)(x)
        x = Conv2D(filters=filters,
                   kernel_size=kernel_size,
                   strides=strides,
                   padding='same')(x)

    x = Flatten()(x)
    x = Dense(1)(x)
    x = Activation('sigmoid')(x)
    # input is conditioned by labels
    discriminator = Model([inputs, labels], x, name='discriminator')
    return discriminator

def train(models, data, params):
   
    generator, discriminator, adversarial = models
    x_train, y_train = data
    batch_size, latent_size, train_steps, num_labels, model_name = params
    save_interval = 500
    noise_input = np.random.uniform(-1.0, 1.0, size=[16, latent_size])
    noise_class = np.eye(num_labels)[np.arange(0, 16) % num_labels] 
    train_size = x_train.shape[0] 

    print(model_name,
          "Labels for generated images: ",
          np.argmax(noise_class, axis=1))

    for i in range(train_steps):
       
        rand_indexes = np.random.randint(0, train_size, size=batch_size)
        real_images = x_train[rand_indexes]
       
        real_labels = y_train[rand_indexes]
      
        noise = np.random.uniform(-1.0,
                                  1.0,
                                  size=[batch_size, latent_size])
       
        fake_labels = np.eye(num_labels)[np.random.choice(num_labels,
                                                          batch_size)]

        fake_images = generator.predict([noise, fake_labels])
        x = np.concatenate((real_images, fake_images))
        labels = np.concatenate((real_labels, fake_labels))

       
        y[batch_size:, :] = 0.0
        loss, acc = discriminator.train_on_batch([x, labels], y)
        log = "%d: [discriminator loss: %f, acc: %f]" % (i, loss, acc)

             
        noise = np.random.uniform(-1.0,
                                  1.0,
                                  size=[batch_size, latent_size])
        fake_labels = np.eye(num_labels)[np.random.choice(num_labels,
                                                          batch_size)]
        y = np.ones([batch_size, 1])
        
        loss, acc = adversarial.train_on_batch([noise, fake_labels], y)
        log = "%s [adversarial loss: %f, acc: %f]" % (log, loss, acc)
        print(log)
        if (i + 1) % save_interval == 0:
            plot_images(generator,
                        noise_input=noise_input,
                        noise_class=noise_class,
                        show=False,
                        step=(i + 1),
                        model_name=model_name)
    
    generator.save(model_name + ".h5")


def plot_images(generator,
                noise_input,
                noise_class,
                show=False,
                step=0,
                model_name="gan"):
    """Generate fake images and plot them
    For visualization purposes, generate fake images
    then plot them in a square grid
    Arguments:
        generator (Model): The Generator Model for fake images generation
        noise_input (ndarray): Array of z-vectors
        show (bool): Whether to show plot or not
        step (int): Appended to filename of the save images
        model_name (string): Model name
    """
    os.makedirs(model_name, exist_ok=True)
    filename = os.path.join(model_name, "%05d.png" % step)
    images = generator.predict([noise_input, noise_class])
    print(model_name , " labels for generated images: ", np.argmax(noise_class, axis=1))
    plt.figure(figsize=(2.2, 2.2))
    num_images = images.shape[0]
    image_size = images.shape[1]
    rows = int(math.sqrt(noise_input.shape[0]))
    for i in range(num_images):
        plt.subplot(rows, rows, i + 1)
        image = np.reshape(images[i], [image_size, image_size])
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.savefig(filename)
    if show:
        plt.show()
    else:
        plt.close('all')


def build_and_train_models():
    # load MNIST dataset
    (x_train, y_train), (_, _) = mnist.load_data()

    # reshape data for CNN as (28, 28, 1) and normalize
    image_size = x_train.shape[1]
    x_train = np.reshape(x_train, [-1, image_size, image_size, 1])
    x_train = x_train.astype('float32') / 255

    num_labels = np.amax(y_train) + 1
    y_train = to_categorical(y_train)

    model_name = "cgan_mnist"
    # network parameters
    # the latent or z vector is 100-dim
    latent_size = 100
    batch_size = 64
    train_steps = 40000
    lr = 2e-4
    decay = 6e-8
    input_shape = (image_size, image_size, 1)
    label_shape = (num_labels, )

    # build discriminator model
    inputs = Input(shape=input_shape, name='discriminator_input')
    labels = Input(shape=label_shape, name='class_labels')

    discriminator = build_discriminator(inputs, labels, image_size)
    # [1] or original paper uses Adam, 
    # but discriminator converges easily with RMSprop
    optimizer = RMSprop(lr=lr, decay=decay)
    discriminator.compile(loss='binary_crossentropy',
                          optimizer=optimizer,
                          metrics=['accuracy'])
    discriminator.summary()

    # build generator model
    input_shape = (latent_size, )
    inputs = Input(shape=input_shape, name='z_input')
    generator = build_generator(inputs, labels, image_size)
    generator.summary()

    # build adversarial model = generator + discriminator
    optimizer = RMSprop(lr=lr*0.5, decay=decay*0.5)
    # freeze the weights of discriminator during adversarial training
    discriminator.trainable = False
    outputs = discriminator([generator([inputs, labels]), labels])
    adversarial = Model([inputs, labels],
                        outputs,
                        name=model_name)
    adversarial.compile(loss='binary_crossentropy',
                        optimizer=optimizer,
                        metrics=['accuracy'])
    adversarial.summary()

    # train discriminator and adversarial networks
    models = (generator, discriminator, adversarial)
    data = (x_train, y_train)
    params = (batch_size, latent_size, train_steps, num_labels, model_name)
    train(models, data, params)


def test_generator(generator, class_label=None):
    noise_input = np.random.uniform(-1.0, 1.0, size=[16, 100])
    step = 0
    if class_label is None:
        num_labels = 10
        noise_class = np.eye(num_labels)[np.random.choice(num_labels, 16)]
    else:
        noise_class = np.zeros((16, 10))
        noise_class[:,class_label] = 1
        step = class_label

    plot_images(generator,
                noise_input=noise_input,
                noise_class=noise_class,
                show=True,
                step=step,
                model_name="test_outputs")


if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    help_ = "Load generator h5 model with trained weights"
    parser.add_argument("-g", "--generator", help=help_)
    help_ = "Specify a specific digit to generate"
    parser.add_argument("-d", "--digit", type=int, help=help_)
    args = parser.parse_args()
    if args.generator:
        generator = load_model(args.generator)
        class_label = None
        if args.digit is not None:
            class_label = args.digit
        test_generator(generator, class_label)
    else:
        build_and_train_models()

usage: ipykernel_launcher.py [-h] [-g GENERATOR] [-d DIGIT]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-a6c5beb1-e4e3-4cbf-be6b-8ea8dda381df.json


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
